In [14]:

import pandas as pd
import numpy as np
from fbprophet import Prophet
import scipy
import math
from statsmodels.tsa.stattools import adfuller
import psycopg2 
import pandas.io.sql as psql
import pandasql as ps
import seaborn as sns
from matplotlib import pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import plotly.express as px
import plotly.io as plt_io
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
import plotly.graph_objs as go
import chart_studio.plotly as py
import plotly.express as px
# Cufflinks wrapper on plotly
import cufflinks

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
import plotly.offline as py
from plotly.offline import init_notebook_mode
from datetime import datetime 
# Set global theme
cufflinks.set_config_file(world_readable=True, theme='pearl')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster


In [2]:
conn = psycopg2.connect(
    host="d-an-test.cygqmbt4gi1l.eu-central-1.rds.amazonaws.com",
    database="postgres",
    user="srijan_goyal",
    password="a{NXDK\wv8AW/>g9")

In [142]:
app_campaigns = pd.read_sql_query('SELECT * FROM fbt_conversions_app',con=conn)
web_campaigns = pd.read_sql_query('SELECT * FROM fbt_conversions_web',con=conn)
campaigns = pd.read_sql_query('SELECT * FROM fbt_facebook_campaigns',con=conn)
installs = pd.read_sql_query('SELECT * FROM fbt_installs',con=conn)
web_sessions = pd.read_sql_query('SELECT * FROM fbt_web_sessions',con=conn)
web_user =  pd.read_sql_query('SELECT * FROM fbt_web_user_mapping',con=conn)

# Data Checks

In [4]:
app_campaigns[app_campaigns.duplicated()]  # Shows the duplicate values if any
app_campaigns.isna().sum() # Checks for na values

,sk_user,purchased_at,subscription_id,platform,sub_region,revenue


sk_user            0
purchased_at       0
subscription_id    0
platform           0
sub_region         0
revenue            0
dtype: int64

In [5]:
web_campaigns[web_campaigns.duplicated()]  # Shows the duplicate values if any
web_campaigns.isna().sum() # Checks for na values

,sk_user,purchased_at,subscription_id,platform,sub_region,revenue


sk_user            0
purchased_at       0
subscription_id    0
platform           0
sub_region         0
revenue            0
dtype: int64

In [6]:
campaigns[campaigns.duplicated()]  # Shows the duplicate values if any
campaigns.isna().sum() # Checks for na values

,campaign_name,campaign_id,spend_date,sub_region,impressions,spend


campaign_name    0
campaign_id      0
spend_date       0
sub_region       0
impressions      0
spend            0
dtype: int64

In [7]:
installs[installs.duplicated()]  # Shows the duplicate values if any
installs.isna().sum() # Checks for na values

,sk_user,install_timestamp,network,sub_region,campaign_id
12,85616463,2020-11-04 13:00:28,Facebook & Instagram,Northern & Western Europe,279755
40,5758257252,2020-11-09 20:02:55,Facebook & Instagram,Northern America,279755
45,5757173863,2020-11-01 02:38:12,Facebook & Instagram,South & Central America,279755
77,5749044149,2020-08-14 21:07:12,Facebook & Instagram,Northern & Western Europe,279755
145,5764533231,2020-12-22 10:02:53,Facebook & Instagram,Northern & Western Europe,279755
...,...,...,...,...,...
466759,,2020-12-21 21:00:42,Facebook & Instagram,Northern America,None
466761,,2021-01-02 13:51:55,Facebook & Instagram,Northern America,None
466772,,2021-01-02 12:20:48,Facebook & Instagram,Southern & Eastern Europe,None
466773,,2021-01-02 15:02:49,Facebook & Instagram,Northern America,None


sk_user                  0
install_timestamp        0
network                  0
sub_region               0
campaign_id          19803
dtype: int64

#### We can see that Installs dataframe has 5817 duplicate values
#### We remove these duplicate values


In [8]:
installs = installs.drop_duplicates()
installs

,sk_user,install_timestamp,network,sub_region,campaign_id
0,5733122290,2020-10-05 01:49:18,Facebook & Instagram,South & Central America,339703
1,5760231103,2020-11-22 17:58:43,Facebook & Instagram,Southern & Eastern Europe,279755
2,5765952637,2020-12-27 11:27:10,Facebook & Instagram,Northern & Western Europe,279755
3,5752380654,2020-09-28 10:14:36,Facebook & Instagram,Southern & Eastern Europe,279755
4,5754228389,2020-10-06 13:57:30,Facebook & Instagram,Northern & Western Europe,279755
...,...,...,...,...,...
468391,5771034024,2021-01-12 20:52:04,Facebook & Instagram,Southern & Eastern Europe,None
468392,5761609197,2020-12-01 15:21:00,Facebook & Instagram,Northern America,None
468393,5765951462,2020-12-27 22:02:21,Facebook & Instagram,Southern & Eastern Europe,None
468394,5765532092,2020-12-25 08:52:57,Facebook & Instagram,Northern America,None


In [9]:
web_sessions[web_sessions.duplicated()]  # Shows the duplicate values if any
web_sessions.isna().sum() # Checks for na values

,sk_web_user,session_id,session_started_timestamp,network,campaign_id,sub_region


sk_web_user                  0
session_id                   0
session_started_timestamp    0
network                      0
campaign_id                  0
sub_region                   0
dtype: int64

In [10]:
web_user[web_user.duplicated()]  # Shows the duplicate values if any
web_user.isna().sum() # Checks for na values

,sk_user,domain_userid,sk_web_user


sk_user          0
domain_userid    0
sk_web_user      0
dtype: int64

#### We don't have any more missing values 
#### The Data Check is done

## ROAS

### ROAS by Campaign id and region

In [167]:
web = pd.merge(web_user,web_sessions, on= 'sk_web_user', how = 'left')
web = web.drop(columns = ['session_id','network','domain_userid','sk_web_user'])
web_roas = pd.merge(web_campaigns, web, how = 'left', on = 'sk_user')
web_roas = web_roas.replace(np.nan,'None').groupby(['campaign_id', 'sub_region_x'])['revenue'].sum().reset_index()

campaigns_web = campaigns[campaigns.campaign_id == '280020'].groupby(['campaign_id','sub_region'])['impressions','spend'].sum().reset_index()
web_final = pd.merge(web_roas,campaigns_web, left_on=['sub_region_x','campaign_id'], right_on=['sub_region','campaign_id'] , how = 'left').drop(columns = 'sub_region')
web_final['ROAS'] = (web_final['revenue']/web_final['spend']).round(2)
web_final = web_final.rename(columns={'sub_region_x': 'sub_region'})

In [169]:
campaigns_sub = campaigns.groupby(['campaign_id','sub_region'])['impressions','spend'].sum().reset_index()
transaction = pd.concat([app_campaigns,web_campaigns])
transaction = transaction.groupby(['sk_user','sub_region'])['revenue'].sum().reset_index()
install_transaction = pd.merge(transaction,installs, how  = 'left', on = ['sk_user','sub_region'])
all_transaction_install = install_transaction.groupby(['campaign_id','sub_region'])['revenue'].sum().reset_index()
ROAS = pd.merge(all_transaction_install,campaigns_sub, how  = 'inner', on = ['campaign_id','sub_region'])
ROAS['ROAS'] = (ROAS['revenue']/ROAS['spend']).round(2)
ROAS = pd.concat([ROAS,web_final]).reset_index(drop = True)

fig = px.treemap(ROAS, path = ['sub_region','campaign_id'],values = 'ROAS',
                    color_discrete_sequence=['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'],
                    title = 'ROAS by Campaign id and region')
fig.update_traces(textposition='middle center', textinfo = "label+value",selector=dict(type='treemap'),textfont_size=14)  


282849.51000000135

### ROAS by Campaign id 

In [181]:
web = pd.merge(web_user,web_sessions, on= 'sk_web_user', how = 'left')
web = web.drop(columns = ['session_id','network','domain_userid','sk_web_user'])
web_roas = pd.merge(web_campaigns, web, how = 'left', on = 'sk_user')
web_roas = web_roas.replace(np.nan,'None').groupby(['campaign_id'])['revenue'].sum().reset_index()

campaigns_web = campaigns[campaigns.campaign_id == '280020'].groupby(['campaign_id','campaign_name'])['impressions','spend'].sum().reset_index()
web_final = pd.merge(web_roas,campaigns_web, left_on=['campaign_id'], right_on=['campaign_id'] , how = 'left')
web_final['ROAS'] = (web_final['revenue']/web_final['spend']).round(2)


In [182]:
campaigns_sub = campaigns.groupby(['campaign_id','campaign_name'])['impressions','spend'].sum().reset_index()
transaction = pd.concat([app_campaigns,web_campaigns])
transaction = transaction.groupby(['sk_user'])['revenue'].sum().reset_index()
install_transaction = pd.merge(transaction,installs, how  = 'left', on = ['sk_user'])
all_transaction_install = install_transaction.groupby(['campaign_id'])['revenue'].sum().reset_index()
ROAS = pd.merge(all_transaction_install,campaigns_sub, how  = 'inner', on = ['campaign_id'])
ROAS['ROAS'] = (ROAS['revenue']/ROAS['spend']).round(2)
ROAS = pd.concat([ROAS,web_final]).reset_index(drop = True)
ROAS = ROAS.dropna()
ROAS
fig = px.bar(ROAS, x='campaign_name', y='ROAS',text = 'ROAS',color = 'campaign_name',
             color_discrete_sequence= ['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'])
fig = fig.update_layout(xaxis={'categoryorder':'total descending'},showlegend=False)
fig.update_traces(texttemplate='%{text:.2f}x', textposition='inside')

,campaign_id,revenue,campaign_name,impressions,spend,ROAS
0,279755,156095.24,ios_campaign_world,39687849.0,155666.0,1.00
1,339703,44868.93,android_campaign_world,19848437.0,46391.0,0.97
2,379581,24295.44,ios_campaign_north_america,4631901.0,63239.0,0.38
3,280020,50997.98,web_campaign_world,11629746.0,68982.0,0.74


### ROAS by sub_region

In [197]:
web = pd.merge(web_user,web_sessions, on= 'sk_web_user', how = 'left')
web = web.drop(columns = ['session_id','network','domain_userid','sk_web_user'])
web_roas = pd.merge(web_campaigns, web, how = 'left', on = 'sk_user')
web_roas = web_roas.replace(np.nan,'None').groupby(['sub_region_x'])['revenue'].sum().reset_index()

campaigns_web = campaigns[campaigns.campaign_id == '280020'].groupby(['sub_region'])['impressions','spend'].sum().reset_index()
web_final = pd.merge(web_roas,campaigns_web, left_on=['sub_region_x'], right_on=['sub_region'] , how = 'left')
web_final['ROAS'] = (web_final['revenue']/web_final['spend']).round(2)
web_final.drop(columns=['sub_region_x'], inplace=True)
web_final = web_final[['sub_region','revenue','impressions','spend','ROAS']]

In [198]:
campaigns_sub = campaigns.groupby(['sub_region'])['impressions','spend'].sum().reset_index()
transaction = pd.concat([app_campaigns,web_campaigns])

transaction = transaction.groupby(['sub_region'])['revenue'].sum().reset_index()

ROAS = pd.merge(transaction,campaigns_sub, how  = 'inner', on = ['sub_region'])

ROAS = pd.concat([ROAS,web_final]).reset_index(drop = True)
ROAS = ROAS.groupby(['sub_region'])['revenue','impressions','spend'].sum().reset_index()
ROAS['ROAS'] = (ROAS['revenue']/ROAS['spend']).round(2)
ROAS
fig = px.bar(ROAS, x='sub_region', y='ROAS',text = 'ROAS',color = 'sub_region',
             color_discrete_sequence= ['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'])
fig = fig.update_layout(xaxis={'categoryorder':'total descending'},showlegend=False)
fig = fig.update_xaxes(tickangle=0)
fig.update_traces(texttemplate='%{text:.2f}x', textposition='inside')

,sub_region,revenue,impressions,spend,ROAS
0,Australasia & Asia,39038.24,16340208,29565.0,1.32
1,Northern & Western Europe,137890.64,15586252,108656.0,1.27
2,Northern America,154230.39,11648790,179247.0,0.86
3,South & Central America,38298.72,28305945,38459.0,1.00
4,Southern & Eastern Europe,52619.33,15546484,47333.0,1.11


### ROAS Evolution

In [32]:
campaigns['spend_date'] = pd.to_datetime(campaigns['spend_date'])
campaigns['spend_date_dt'] = campaigns['spend_date'].dt.date
campaigns_sub = campaigns.groupby(['spend_date_dt'])['impressions','spend'].sum().reset_index()

transaction = pd.concat([app_campaigns,web_campaigns])
transaction['purchased_at'] = pd.to_datetime(transaction['purchased_at'])
transaction['purchased_at_date'] = transaction['purchased_at'].dt.date

transaction = transaction.groupby(['purchased_at_date'])['revenue'].sum().reset_index()

ROAS = pd.merge(transaction,campaigns_sub, how  = 'inner', right_on = ['spend_date_dt'],left_on = ['purchased_at_date'] )
#ROAS = pd.concat([ROAS,web_final]).reset_index(drop = True)
ROAS = ROAS.groupby(['purchased_at_date'])['revenue','impressions','spend'].sum().reset_index()
ROAS['ROAS'] = (ROAS['revenue']/ROAS['spend']).round(2)
fig = go.Figure(data=go.Scatter(x=ROAS.purchased_at_date, y=ROAS.ROAS, mode='lines+markers', line = {'color' : '#1c3d5e'},
                                text = ROAS.ROAS,texttemplate='%{text:.1f}', textposition='top center' ))
fig = fig.update_layout(
    title="ROAS evolution",
    xaxis_title="Date",
    yaxis_title="ROAS")
fig = fig.add_annotation(x='2020-11-30', y=0.76,
            text="Highest ROAS (0.76x)",
            showarrow=True,
        font=dict(
            family="Roboto, monospace, bold",
            size=16,
            color="#1c3d5e"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#c0d6df",
        opacity=0.9)
fig.add_annotation(x='2020-11-17', y=0.33,
            text="Lowest ROAS (0.33x)",
            showarrow=True,
        font=dict(
            family="Roboto, monospace, bold",
            size=16,
            color="#1c3d5e"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#c0d6df",
        opacity=1)

## Conversion Rate

### How many percentage installers make a purchase for each campaign_id by region

In [5]:
installers= installs.groupby(['campaign_id','sub_region','sk_user'])['sk_user'].count().rename('no_of_installers').reset_index()
web_app = pd.concat([app_campaigns,web_campaigns])

all_web_app=pd.merge(web_app,installers, how  = 'right', on = ['sk_user','sub_region'])
installer_count= all_web_app.groupby(['campaign_id','sub_region'])['no_of_installers'].sum().reset_index()

all_web_app_users=pd.merge(web_app,installers, how  = 'left', on = ['sk_user','sub_region'])
all_web_app_users= all_web_app_users.groupby(['campaign_id','sub_region'])['sk_user'].count().rename('no_of_users').reset_index()
cvr = pd.merge(installer_count,all_web_app_users, how = 'inner')
cvr['CVR'] = (cvr['no_of_users']/cvr['no_of_installers']* 100).round(2)
cvr


,campaign_id,sub_region,no_of_installers,no_of_users,CVR
0,279755,Australasia & Asia,23951,473,1.97
1,279755,Northern & Western Europe,58859,2541,4.32
2,279755,Northern America,38640,1633,4.23
3,279755,South & Central America,50131,831,1.66
4,279755,Southern & Eastern Europe,32116,587,1.83
5,339703,Australasia & Asia,37819,120,0.32
6,339703,Northern & Western Europe,39258,896,2.28
7,339703,Northern America,9885,439,4.44
8,339703,South & Central America,127350,330,0.26
9,339703,Southern & Eastern Europe,22125,197,0.89


In [7]:
fig = px.treemap(cvr, path = ['sub_region','campaign_id'],values = 'CVR',
                    color_discrete_sequence=['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'],
                    title = 'CVR by Campaign id and region')
fig.update_traces(textposition='middle center', textinfo = "label+value",selector=dict(type='treemap'),textfont_size=14)  



#### Since we can't attribute web campaigns to installlers, we cannot calculate the CTR for campaign 280020

### How many percentage installers make a purchase for each campaign_id

In [107]:
campaigns_sub = campaigns.groupby(['campaign_id','campaign_name','sub_region'])['impressions','spend'].sum().reset_index()
installers= installs.groupby(['campaign_id','sub_region','sk_user'])['sk_user'].count().rename('no_of_installers').reset_index()
camp_installers= pd.merge(campaigns_sub,installers, on = ['campaign_id','sub_region'], how ='right')

installers= camp_installers.groupby(['campaign_id','campaign_name','sub_region','sk_user'])['sk_user'].count().rename('no_of_installers').reset_index()
web_app = pd.concat([app_campaigns,web_campaigns])

all_web_app=pd.merge(web_app,installers, how  = 'right', on = ['sk_user','sub_region'])
installer_count= all_web_app.groupby(['campaign_id','campaign_name'])['no_of_installers'].sum().reset_index()

all_web_app_users=pd.merge(web_app,installers, how  = 'left', on = ['sk_user','sub_region'])
all_web_app_users= all_web_app_users.groupby(['campaign_id','campaign_name'])['sk_user'].count().rename('no_of_users').reset_index()
cvr = pd.merge(installer_count,all_web_app_users, how = 'inner')
cvr['CVR'] = (cvr['no_of_users']/cvr['no_of_installers']* 100).round(2)
cvr


,campaign_id,campaign_name,no_of_installers,no_of_users,CVR
0,279755,ios_campaign_world,197605,6065,3.07
1,339703,android_campaign_world,230201,1982,0.86
2,379581,ios_campaign_north_america,8260,968,11.72


In [108]:
fig = px.bar(cvr, x='campaign_name', y='CVR',text = 'CVR',color = 'campaign_name',
             color_discrete_sequence= ['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'])
fig = fig.update_layout(xaxis={'categoryorder':'total descending'},showlegend=False)
fig = fig.update_xaxes(tickangle=0)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='inside')

### How many percentage installers make a purchase for each sub_region

In [109]:
campaigns_sub = campaigns.groupby(['sub_region'])['impressions','spend'].sum().reset_index()
installers= installs.groupby(['campaign_id','sub_region','sk_user'])['sk_user'].count().rename('no_of_installers').reset_index()
camp_installers= pd.merge(campaigns_sub,installers, on = ['sub_region'], how ='right')

installers= camp_installers.groupby(['sub_region','sk_user'])['sk_user'].nunique().rename('no_of_installers').reset_index()
web_app = pd.concat([app_campaigns,web_campaigns])

all_web_app=pd.merge(web_app,installers, how  = 'right', on = ['sk_user','sub_region'])
installer_count= all_web_app.groupby(['sub_region'])['no_of_installers'].sum().reset_index()

all_web_app_users=pd.merge(web_app,installers, how  = 'left', on = ['sk_user','sub_region'])
all_web_app_users= all_web_app_users.groupby(['sub_region'])['sk_user'].count().rename('no_of_users').reset_index()
cvr = pd.merge(installer_count,all_web_app_users, how = 'inner')
cvr['CVR'] = (cvr['no_of_users']/cvr['no_of_installers']* 100).round(2)
cvr


,sub_region,no_of_installers,no_of_users,CVR
0,Australasia & Asia,61822,1187,1.92
1,Northern & Western Europe,102093,5168,5.06
2,Northern America,60888,5211,8.56
3,South & Central America,173455,1595,0.92
4,Southern & Eastern Europe,57469,1802,3.14


In [110]:
fig = px.bar(cvr, x='sub_region', y='CVR',text = 'CVR',color = 'sub_region',
             color_discrete_sequence= ['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'])
fig = fig.update_layout(xaxis={'categoryorder':'total descending'},showlegend = False)
fig = fig.update_xaxes(tickangle=0)
fig.update_traces(texttemplate='%{text:.0f}%', textposition='inside')




## CPM

### Cost per 1000 impressions for each campaign_id by sub_region

In [9]:
CPM=campaigns.groupby(['campaign_id','sub_region'])['impressions','spend'].sum().reset_index()
CPM['CPM']= ((CPM['spend'] * 1000)/CPM['impressions']).round(2)
CPM

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,campaign_id,sub_region,impressions,spend,CPM
0,279755,Australasia & Asia,3494920,12651.0,3.62
1,279755,Northern & Western Europe,8572399,52304.0,6.10
2,279755,Northern America,4272460,49692.0,11.63
3,279755,South & Central America,15155003,23106.0,1.52
4,279755,Southern & Eastern Europe,8193067,17913.0,2.19
5,280020,Australasia & Asia,4677048,7213.0,1.54
6,280020,Northern & Western Europe,1226373,16404.0,13.38
7,280020,Northern America,956722,28836.0,30.14
8,280020,South & Central America,2373242,4623.0,1.95
9,280020,Southern & Eastern Europe,2396361,11906.0,4.97


In [10]:
fig = px.treemap(CPM, path = ['sub_region','campaign_id'],values = 'CPM',
                    color_discrete_sequence=['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'],
                    title = 'CPM by Campaign id and region')
fig.update_traces(textposition='middle center', textinfo = "label+value",selector=dict(type='treemap'),textfont_size=16)  



### CPM for each campaign_id 

In [199]:
CPM=campaigns.groupby(['campaign_id','campaign_name'])['impressions','spend'].sum().reset_index()
CPM['CPM']= ((CPM['spend'] * 1000)/CPM['impressions']).round(2)
CPM
fig = px.bar(CPM, x='campaign_name', y='CPM',text = 'CPM',color = 'campaign_name',
             color_discrete_sequence= ['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'])
fig = fig.update_layout(xaxis={'categoryorder':'total ascending'},showlegend = False)
fig.update_traces(texttemplate='€%{text:.1f}', textposition='inside')

#fig.update_xaxes(type='category')

,campaign_id,campaign_name,impressions,spend,CPM
0,279755,ios_campaign_world,39687849,155666.0,3.92
1,280020,web_campaign_world,11629746,68982.0,5.93
2,339703,android_campaign_world,19848437,46391.0,2.34
3,379581,ios_campaign_north_america,4631901,63239.0,13.65


### CPM for each sub region

In [111]:
CPM=campaigns.groupby(['sub_region'])['impressions','spend'].sum().reset_index()
CPM['CPM']= ((CPM['spend'] * 1000)/CPM['impressions']).round(2)
CPM
fig = px.bar(CPM, x='sub_region', y='CPM',text = 'CPM',color = 'sub_region',
             color_discrete_sequence= ['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'])
fig = fig.update_layout(xaxis={'categoryorder':'total ascending'},showlegend = False)
fig = fig.update_xaxes(tickangle=0)
fig.update_traces(texttemplate='€%{text:.1f}', textposition='inside')




,sub_region,impressions,spend,CPM
0,Australasia & Asia,11663160,22352.0,1.92
1,Northern & Western Europe,14359879,92252.0,6.42
2,Northern America,10692068,150411.0,14.07
3,South & Central America,25932703,33836.0,1.30
4,Southern & Eastern Europe,13150123,35427.0,2.69


## CPI

### Cost per install for each campaign_id by sub_region

In [13]:
campaigns_sub = campaigns.groupby(['campaign_id','sub_region'])['spend'].sum().reset_index()
installers= installs.groupby(['campaign_id','sub_region'])['sk_user'].nunique().rename('no_of_installers').reset_index()
camp_installers= pd.merge(campaigns_sub,installers, on =['campaign_id','sub_region'], how ='right')
camp_installers['CPI'] = (camp_installers['spend']/camp_installers['no_of_installers']).round(2)
camp_installers
fig = px.treemap(camp_installers, path = ['sub_region','campaign_id'],values = 'CPI',
                    color_discrete_sequence=['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'],
                    title = 'CPI by Campaign id and region')
fig.update_traces(textposition='middle center', textinfo = "label+value",selector=dict(type='treemap'),textfont_size=16)  


,campaign_id,sub_region,spend,no_of_installers,CPI
0,279755,Australasia & Asia,12651.0,23068,0.55
1,279755,Northern & Western Europe,52304.0,57424,0.91
2,279755,Northern America,49692.0,37170,1.34
3,279755,South & Central America,23106.0,48576,0.48
4,279755,Southern & Eastern Europe,17913.0,31357,0.57
5,339703,Australasia & Asia,2488.0,36725,0.07
6,339703,Northern & Western Europe,23544.0,38634,0.61
7,339703,Northern America,8644.0,9637,0.90
8,339703,South & Central America,6107.0,123286,0.05
9,339703,Southern & Eastern Europe,5608.0,21850,0.26


### Cost per install for each campaign_id 

In [14]:
campaigns_sub = campaigns.groupby(['campaign_id','campaign_name'])['spend'].sum().reset_index()
installers= installs.groupby(['campaign_id'])['sk_user'].nunique().rename('no_of_installers').reset_index()
camp_installers= pd.merge(campaigns_sub,installers, on = ['campaign_id'], how ='right')
camp_installers['CPI'] = (camp_installers['spend']/camp_installers['no_of_installers']).round(2)
camp_installers
fig = px.bar(camp_installers, x='campaign_name', y='CPI',text = 'CPI',color_discrete_sequence = ['#1c3d5e'])
fig.update_xaxes(type='category')

,campaign_id,campaign_name,spend,no_of_installers,CPI
0,279755,ios_campaign_world,155666.0,197591,0.79
1,339703,android_campaign_world,46391.0,230128,0.20
2,379581,ios_campaign_north_america,63239.0,8314,7.61


#### Since we can't attribute web campaigns to installlers, we cannot calculate the CPI for campaign 280020

### Cost per install for each sub_region

In [15]:
campaigns_sub = campaigns.groupby(['sub_region'])['spend'].sum().reset_index()
installers= installs.groupby(['sub_region'])['sk_user'].nunique().rename('no_of_installers').reset_index()
camp_installers= pd.merge(campaigns_sub,installers, on = ['sub_region'], how ='right')
camp_installers['CPI'] = (camp_installers['spend']/camp_installers['no_of_installers']).round(2)
camp_installers
fig = px.bar(camp_installers, x='sub_region', y='CPI',text = 'CPI',color_discrete_sequence = ['#1c3d5e'])
fig.update_xaxes(type='category')

,sub_region,spend,no_of_installers,CPI
0,Australasia & Asia,22352.0,61763,0.36
1,Northern & Western Europe,92252.0,101996,0.90
2,Northern America,150411.0,60760,2.48
3,South & Central America,33836.0,173400,0.20
4,Southern & Eastern Europe,35427.0,57384,0.62


## CPA (Cost per action)

### CPA by Campaign id and sub region

In [62]:
campaigns_sub = campaigns.groupby(['campaign_id','sub_region'])['spend'].sum().reset_index()
web_app = pd.concat([app_campaigns,web_campaigns])
users = pd.merge(web_app,installs, on = ['sk_user','sub_region'], how ='inner')
users_sub = users.groupby(['campaign_id','sub_region'])['sk_user'].nunique().rename('no_of_users').reset_index()
cpa = pd.merge(campaigns_sub,users_sub, on = ['campaign_id','sub_region'], how ='inner')
cpa['CPA'] = (cpa['spend']/cpa['no_of_users']).round(2)
cpa
fig = px.treemap(cpa, path = ['sub_region','campaign_id'],values = 'CPA',
                    color_discrete_sequence=['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'],
                    title = 'CPA by Campaign id and region')
fig.update_traces(textposition='middle center', textinfo = "label+value",selector=dict(type='treemap'),textfont_size=16)  



,campaign_id,sub_region,spend,no_of_users,CPA
0,279755,Australasia & Asia,12651.0,472,26.80
1,279755,Northern & Western Europe,52304.0,2537,20.62
2,279755,Northern America,49692.0,1632,30.45
3,279755,South & Central America,23106.0,828,27.91
4,279755,Southern & Eastern Europe,17913.0,586,30.57
5,339703,Australasia & Asia,2488.0,116,21.45
6,339703,Northern & Western Europe,23544.0,859,27.41
7,339703,Northern America,8644.0,432,20.01
8,339703,South & Central America,6107.0,322,18.97
9,339703,Southern & Eastern Europe,5608.0,184,30.48


### CPA by Campaign id 

In [101]:
web = pd.merge(web_user,web_sessions, on= 'sk_web_user', how = 'left')
web = web.drop(columns = ['session_id','network','domain_userid','sk_web_user'])
web_roas = pd.merge(web_campaigns, web, how = 'left', on = 'sk_user')
web_roas = web_roas.replace(np.nan,'None').groupby(['campaign_id'])['sk_user'].nunique().rename('no_of_users').reset_index()
#web_roas.sort_values(by = ['campaign_id', 'revenue'],ascending = False)
campaigns_web = campaigns[campaigns.campaign_id == '280020'].groupby(['campaign_id','campaign_name'])['impressions','spend'].sum().reset_index()
web_final = pd.merge(web_roas,campaigns_web, left_on=['campaign_id'], right_on=['campaign_id'] , how = 'left')
web_final['CPA'] = (web_final['spend']/web_final['no_of_users']).round(2)
web_final = web_final[['campaign_id','campaign_name','spend','no_of_users','CPA']]
#web_final.rename(columns={'sub_region_x': 'sub_region'}, inplace=True)

In [102]:
campaigns_sub = campaigns.groupby(['campaign_id','campaign_name'])['spend'].sum().reset_index()
web_app = pd.concat([app_campaigns,web_campaigns])
users = pd.merge(web_app,installs, on = ['sk_user'], how ='inner')
users = users.groupby(['campaign_id'])['sk_user'].nunique().rename('no_of_users').reset_index()
cpa = pd.merge(campaigns_sub,users, on = ['campaign_id'], how ='inner')
cpa['CPA'] = (cpa['spend']/cpa['no_of_users']).round(2)
cpa = pd.concat([cpa,web_final]).reset_index(drop = True).dropna()
fig = px.bar(cpa, x='campaign_name', y='CPA',text = 'CPA',color = 'campaign_name',
             color_discrete_sequence= ['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'])
fig = fig.update_layout(xaxis={'categoryorder':'total ascending'},showlegend = False)
fig.update_traces(texttemplate='€%{text:.1f}', textposition='inside')

### CPA by sub region

In [103]:
web = pd.merge(web_user,web_sessions, on= 'sk_web_user', how = 'left')
web = web.drop(columns = ['session_id','network','domain_userid','sk_web_user'])
web_roas = pd.merge(web_campaigns, web, how = 'left', on = ['sk_user','sub_region'])
web_roas = web_roas.replace(np.nan,'None').groupby(['sub_region'])['sk_user'].nunique().rename('no_of_users').reset_index()
#web_roas.sort_values(by = ['campaign_id', 'revenue'],ascending = False)
campaigns_web = campaigns[campaigns.campaign_id == '280020'].groupby(['sub_region'])['impressions','spend'].sum().reset_index()
web_final = pd.merge(web_roas,campaigns_web, left_on=['sub_region'], right_on=['sub_region'] , how = 'left')
web_final['CPA'] = (web_final['spend']/web_final['no_of_users']).round(2)
web_final = web_final[['sub_region','spend','no_of_users','CPA']]
#web_final.rename(columns={'sub_region_x': 'sub_region'}, inplace=True)

In [104]:
campaigns_sub = campaigns.groupby(['sub_region'])['spend'].sum().reset_index()
web_app = pd.concat([app_campaigns,web_campaigns])
users = pd.merge(web_app,installs, on = ['sk_user','sub_region'], how ='inner')
users_sub = users.groupby(['sub_region'])['sk_user'].nunique().rename('no_of_users').reset_index()
cpa = pd.merge(campaigns_sub,users_sub, on = ['sub_region'], how ='inner')
cpa = pd.concat([cpa,web_final]).reset_index(drop = True)
cpa = cpa.groupby(['sub_region'])['spend','no_of_users'].sum().reset_index()
cpa['CPA'] = (cpa['spend']/cpa['no_of_users']).round(2)
fig = px.bar(cpa, x='sub_region', y='CPA',text = 'CPA',color = 'sub_region',
             color_discrete_sequence= ['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'])
fig = fig.update_layout(xaxis={'categoryorder':'total ascending'},showlegend = False)
fig.update_traces(texttemplate='€%{text:.1f}', textposition='inside')




#### In my view, the best metric to look at to measure performance is ROAS because ROAS is an indicator that measures whether the marketing channel you are using is profitable or not and help advertisers gauge what’s working and how they can improve future efforts

# ARPU

## ARPU by campaign

In [188]:
web = pd.merge(web_user,web_sessions, on= 'sk_web_user', how = 'left')
web = web.drop(columns = ['session_id','network','domain_userid','sk_web_user'])
web_rev = pd.merge(web_campaigns, web, how = 'left', on = 'sk_user')
web_rev = web_rev.replace(np.nan,'None').groupby(['campaign_id'])['revenue'].sum().reset_index()
web_count = pd.merge(web_campaigns, web, how = 'left', on = 'sk_user')
web_count = web_count.replace(np.nan,'None').groupby(['campaign_id'])['sk_user'].nunique().rename('no_of_users').reset_index()
web_final = pd.merge(web_count,web_rev, left_on=['campaign_id'], right_on=['campaign_id'] , how = 'inner')
camp_sub = campaigns[['campaign_id','campaign_name']]
final = pd.merge(camp_sub,web_final).drop_duplicates()


In [189]:
campaigns_sub = campaigns.groupby(['campaign_id','campaign_name','sub_region'])['impressions','spend'].sum().reset_index()
installers= installs.groupby(['campaign_id','sub_region','sk_user'])['sk_user'].count().rename('no_of_installers').reset_index()
camp_installers= pd.merge(campaigns_sub,installers, on = ['campaign_id','sub_region'], how ='right')

installers= camp_installers.groupby(['campaign_id','campaign_name','sub_region','sk_user'])['sk_user'].count().rename('no_of_installers').reset_index()
web_app = pd.concat([app_campaigns,web_campaigns])

all_web_app=pd.merge(web_app,installers, how  = 'inner', on = ['sk_user'])
user_rev= all_web_app.groupby(['campaign_id','campaign_name'])['revenue'].sum().reset_index()
user_count = all_web_app.groupby(['campaign_id','campaign_name'])['sk_user'].nunique().rename('no_of_users').reset_index()
all_web_app_users=pd.merge(user_count,user_rev, how  = 'left', on = ['campaign_id','campaign_name'])
ARPU = pd.concat([all_web_app_users,final])
ARPU['ARPU'] = (ARPU['revenue']/ARPU['no_of_users']).round(2)
fig = px.bar(ARPU, x='campaign_name', y='ARPU',text = 'ARPU',color = 'campaign_name',
             color_discrete_sequence= ['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'])
fig = fig.update_layout(xaxis={'categoryorder':'total descending'},showlegend = False)
fig.update_traces(texttemplate='€%{text:.1f}', textposition='inside')


## ARPU by sub_region

In [191]:
web = pd.merge(web_user,web_sessions, on= 'sk_web_user', how = 'left')
web = web.drop(columns = ['session_id','network','domain_userid','sk_web_user'])
web_arpu = pd.merge(web_campaigns, web, how = 'left', on = ['sk_user','sub_region'])
web_rev = web_arpu.replace(np.nan,'None').groupby(['sub_region'])['revenue'].sum().reset_index()
web_user_final = web_arpu.replace(np.nan,'None').groupby(['sub_region'])['sk_user'].nunique().rename('no_of_users').reset_index()
web_final = pd.merge(web_rev,web_user_final, left_on=['sub_region'], right_on=['sub_region'] , how = 'inner')


In [192]:
web_app = pd.concat([app_campaigns,web_campaigns])
users = web_app.groupby(['sub_region'])['sk_user'].nunique().rename('no_of_users').reset_index()
users_rev = web_app.groupby(['sub_region'])['revenue'].sum().reset_index()
final_users = pd.merge(users,users_rev,on = 'sub_region', how = 'inner')
ARPU = pd.concat([users_rev,web_final]).reset_index(drop = True)
ARPU = ARPU.groupby('sub_region')['no_of_users','revenue'].sum().reset_index()
ARPU['ARPU'] = (ARPU['revenue']/ARPU['no_of_users']).round(2)
fig = px.bar(ARPU, x='sub_region', y='ARPU',text = 'ARPU',color = 'sub_region',
             color_discrete_sequence= ['#1c3d5e','#50bf97','#fcc866','#e3655b','#c0d6df'])
fig = fig.update_layout(xaxis={'categoryorder':'total descending'},showlegend = False)
fig.update_traces(texttemplate='€%{text:.1f}', textposition='inside')


## Average Customer Lifetime Value

In [88]:
users = web_app.sk_user.nunique()
revenue = web_app.revenue.sum()
LTV = (revenue/users).round(2)
LTV

24.87

## Forecast

In [112]:
all_purchase = pd.concat([app_campaigns,web_campaigns])
all_purchase['purchased_at'] = pd.to_datetime(all_purchase['purchased_at'])
all_purchase['purchased_at_date'] = all_purchase['purchased_at'].dt.date
time_series = all_purchase.groupby(['purchased_at_date'])['revenue'].sum().reset_index()
time_series['dt'] = pd.to_datetime(time_series['purchased_at_date'])
time_series = time_series.sort_values(by='dt')

import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=time_series.purchased_at_date, y=time_series.revenue, mode='lines+markers', line = {'color' : '#1c3d5e'},
                                text = time_series.revenue,texttemplate='€%{text:.1f}', textposition='top center' ))
fig = fig.update_layout(
    title="Revenue evolution",
    xaxis_title="Date",
    yaxis_title="Revenue")
fig = fig.add_annotation(x='2020-12-27', y=10858,
            text="Highest Revenue (€10.85K)",
            showarrow=True,
        font=dict(
            family="Roboto, monospace, bold",
            size=16,
            color="#1c3d5e"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#c0d6df",
        opacity=0.9)
fig.add_annotation(x='2021-02-17', y=95.95,
            text="Lowest Revenue (€95)",
            showarrow=True,
        font=dict(
            family="Roboto, monospace, bold",
            size=16,
            color="#1c3d5e"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#c0d6df",
        opacity=1)


#### We observe a significant drop in revenue starting from 1st Jan 2021. This needs to be investigated

## Check for stationarity

In [114]:
time_series = time_series.rename(columns={"dt": "ds", "revenue": "y"})
X = time_series['y'].values
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))



ADF Statistic: -0.613927
p-value: 0.867804
Critical Values:
	1%: -3.500
	5%: -2.892
	10%: -2.583


From the ADF test we see that the p-value is much more than 0.05 which means the series is non stationary

In [116]:
m = Prophet(yearly_seasonality=True)
m.fit(time_series)
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)
trace = go.Scatter(
    name = 'Actual revenue',
    mode = 'markers',
    x = list(forecast['ds']),
    y = list(time_series['y']),
    marker=dict(
        color='#FFBAD2',
        line=dict(width=1)
    )
)
trace1 = go.Scatter(
    name = 'Trend',
    mode = 'lines',
    x = list(forecast['ds']),
    y = list(forecast['yhat']),
    marker=dict(
        color='red',
        line=dict(width=3)
    )
)
upper_band = go.Scatter(
    name = 'Upper band',
    mode = 'lines',
    x = list(forecast['ds']),
    y = list(forecast['yhat_upper']),
    line= dict(color='#57b88f'),
    fill = 'tonexty'
)
lower_band = go.Scatter(
    name= 'lower band',
    mode = 'lines',
    x = list(forecast['ds']),
    y = list(forecast['yhat_lower']),
    line= dict(color='#1705ff')
)
tracex = go.Scatter(
    name = 'Actual Revenue',
   mode = 'markers',
   x = list(time_series['ds']),
   y = list(time_series['y']),
   marker=dict(
      color='black',
      line=dict(width=2)
   )
)


data = [tracex, trace1, lower_band, upper_band]

layout = dict(title='Revenue Forecast Using FbProphet',
             xaxis=dict(title = 'Dates', ticklen=2, zeroline=True),
             yaxis=dict(title = 'Revenue', ticklen=2, zeroline=True))

figure=dict(data=data,layout=layout)

init_notebook_mode(connected=True)
py.offline.iplot(figure)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


## Web attribution difficulties

In [80]:
installs.campaign_id.replace(np.nan, 'None', inplace=True)
installs_web_potential = installs[installs['campaign_id']=='None'].reset_index(drop=True)
installs_web_potential['sk_user'] = installs_web_potential['sk_user'].apply(lambda x : np.nan if x.strip()=='' else x)
installs_web_potential.dropna(inplace=True)

In [81]:
installs_web_potential['sk_user'].isin(web_user['sk_user']).value_counts()

False    14891
True      4400
Name: sk_user, dtype: int64

In [82]:
web_installs = pd.merge(installs_web_potential, web_user, on='sk_user', how='left')

In [85]:
x = pd.merge(web_installs, web_sessions, on = 'sk_web_user', how = 'inner')
x = x[['sk_user','install_timestamp','sub_region_x','session_started_timestamp', 'campaign_id_y']]
x['install_session_diff'] = x['install_timestamp'] - x['session_started_timestamp']
y = pd.merge(x,web_campaigns, on = 'sk_user', how = 'left')
y['install_session_diff'] = y['purchased_at'] - y['install_timestamp']

In [86]:
z = pd.merge(x,app_campaigns, on = 'sk_user', how = 'left')
z['install_purchase_diff'] = z['purchased_at'] - z['install_timestamp']
z['session_purchase_diff'] = z['purchased_at'] - z['session_started_timestamp']
z

,sk_user,install_timestamp,sub_region_x,session_started_timestamp,campaign_id_y,install_session_diff,purchased_at,subscription_id,platform,sub_region,revenue,install_purchase_diff,session_purchase_diff
0,5759580196,2020-11-18 18:44:43,Northern America,2020-11-18 18:39:48,280020,0 days 00:04:55,2020-11-19 18:39:48,1257903,App,Northern America,21.11,0 days 23:55:05,1 days 00:00:00
1,5760232807,2020-11-23 03:35:34,Northern America,2020-11-23 03:19:17,280020,0 days 00:16:17,2020-11-24 03:19:17,1258021,App,Northern America,21.72,0 days 23:43:43,1 days 00:00:00
2,5763773280,2020-12-15 07:15:01,Northern America,2020-12-15 07:09:53,280020,0 days 00:05:08,NaT,NaN,NaN,NaN,NaN,NaT,NaT
3,189508662,2019-08-09 12:51:22,South & Central America,2020-11-15 03:42:07,280020,-464 days +09:09:15,2020-11-16 03:42:07,1257822,App,South & Central America,16.00,464 days 14:50:45,1 days 00:00:00
4,5750933663,2021-01-01 16:53:29,Northern America,2020-09-03 16:21:59,280020,120 days 00:31:30,NaT,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4431,5750923385,2020-09-03 19:31:45,Australasia & Asia,2020-09-03 19:23:39,280020,0 days 00:08:06,NaT,NaN,NaN,NaN,NaN,NaT,NaT
4432,82034647,2019-06-23 04:37:59,Northern & Western Europe,2020-08-25 04:06:52,280020,-429 days +00:31:07,NaT,NaN,NaN,NaN,NaN,NaT,NaT
4433,5750436838,2020-08-29 17:51:24,Northern America,2020-08-29 17:48:16,280020,0 days 00:03:08,NaT,NaN,NaN,NaN,NaN,NaT,NaT
4434,5762773977,2020-12-09 14:30:21,Northern America,2020-12-09 14:22:00,280020,0 days 00:08:21,NaT,NaN,NaN,NaN,NaN,NaT,NaT


### We check the values of session_purchase_diff and install_session_diff to appropriately attribute campaigns to web

## Perform AB test on various elements of the app

1. Randomly split the data into control and test<br>
   <br>
2. Define Success Metric for evaluating the test. These may include <br>
   <br>ROAS (Return on Ad spend) = Revenue/Spend
   <br>CPM (Cost per 1000 impressions) = (Spend * 1000)/Impressions
   <br>CR (Conversion rate) = # of Purchases/# of Installers
   <br>CPA (Cost per action) = Spend/ # of Purchasers
   <br>ARPU (Average Revenue per user) = Revenue/ # of Purchasers<br>
   <br>
3. Check the distribution of the these metrics being parametric or Non parametric under the following assumptions<br>

   $H_{0}$ : The distribution is Parametric<br>
   $H_{1}$ : The distribution is non Parametric<br>
   <br>
   We use the Shapiro Wilk's test such that <br>
      $W=\frac{\left(\sum_{i=1}^{n} a_{i} x_{(i)}\right)^{2}}{\sum_{i=1}^{n}\left(x_{i}-\bar{x}\right)^{2}}$
   where <br>
   $x_{i}$ are the ordered random sample values
   $a_{i}$ are constants generated from the covariances, variances and means of the sample (size n) from a normally distributed    sample.<br>
   
   We then check the p-value at 95% confidence interval such that
   If 
       p-value > 0.05 => The Null Hypothesis is accepted and the distribution is Parametric
   Else if
       p-value < 0.05 => The Null Hypothesis is rejected and the distribution is non Parametric
      
   
4. Now, we see if there is a significant difference between the metrics in the control and test group. We formulate our hypothesis under  the  following assumptions<br>
   $H_{0}$ = The control and the test group have the same distribution<br>
   $H_{1}$ = The control and the test group have different distribution<br>
   <br>
   For a parametric distribution, we check this using the Z-test 
     $Z=\frac{m-\mu}{s / \sqrt{n}}$
   where <br>
   m is the mean, $\mu$ is the theoretical value, s is the standatd deviation and n is the sample size <br>
   
   We then check the p-value at 95% confidence interval such that
   If 
       p-value > 0.05 => The Null Hypothesis is accepted and the control and the test group have the same distribution
   Else if
       p-value < 0.05 => The Null Hypothesis is rejected and the control and the test group have different distribution
       
   For a non-parametric distribution, we check this using the Mann Whitney U test<br>
     $\begin{aligned} U_{1} &=R_{1}-\frac{n_{1}\left(n_{1}+1\right)}{2} \\ & \text { or } \\ U_{2} &=R_{2}-\frac{n_{2}\left(n_{2}+1\right)}{2} \end{aligned}$ 
   where <br>
   <br>R is the sum of ranks in the sample, and n is the number of items in the sample<br>
   We then check the p-value at 95% confidence interval such that
   If 
       p-value > 0.05 => The Null Hypothesis is accepted and the control and the test group have the same distribution
   Else if
       p-value < 0.05 => The Null Hypothesis is rejected and the control and the test group have different distribution


There are two possibilities now,<br>
<br>If the **control** and the **test** group have the same distribution then <br>
The A/B test **did not** produce any significant results, we tweak the campaigns again and re-run the test <br>
<br>Else if the **control** and the **test** groups have different distribution then <br> 
We implement the change in the campaigns thereby **increasing** the key metrics of performance measurement and attracting more users to the app in the process

Depending on the outcome of the A/B test we implement additional changes to the app. <br>
Every piece of content that reaches your users via your app must be optimized to its maximum potential. This is especially true for elements that can influence visitor behavior and conversion rate. 